### Data file preparation steps:
    
    * The raw text file from https://afltables.com/afl/stats/biglists/bg3.txt is saved as llist.txt
    * The file llist.txt is then processed through the bash script file a_updater.sh to add commas at the appropriate locations and make other modifications, by typing ./a_updater into Terminal - ready to be imported from within Jupyter.

# Post week 1 of 2018 finals series

In [1]:
#%%html
#<img src="img/ess_stk.png" alt="Drawing" align="left" style="width: 800px;"/>

In [2]:
import os 
os.getcwd() 

'/Users/stevegabriel/data_projects/sports_data/afl_dataexploration'

In [3]:
import pandas as pd

In [4]:
cols = ['seq','date','rnd','team1','gls_1','bhs_1','pts_1','team2','gls_2','bhs_2','pts_2','venue']

#### The file will contain all matches starting from the first ever VFL game in 1987 - more than fifteen thousand games.

In [5]:
upd_round = pd.read_csv("newlist.csv", names = cols)
trans = upd_round

In [6]:
# Modify index to start at 1 
trans.index += 1
trans.tail()

,seq,date,rnd,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,pts_2,venue
15398,15398,26-Aug-2018,R23,St Kilda,14,10,94,North Melbourne,17,15,117,Docklands
15399,15399,6-Sep-2018,QF,Richmond,13,17,95,Hawthorn,9,10,64,M.C.G.
15400,15400,7-Sep-2018,EF,Melbourne,10,15,75,Geelong,6,10,46,M.C.G.
15401,15401,8-Sep-2018,EF,GW Sydney,10,19,79,Sydney,4,6,30,S.C.G.
15402,15402,8-Sep-2018,QF,West Coast,12,14,86,Collingwood,10,10,70,Perth Stadium


#### Shown above, the most four finals matches played on the weekend.

### Adding additional information including state, and addition team names as two letter codes

In [7]:
# Add states
nsw = ['S.C.G.','Sydney Showground','Stadium Australia','Blacktown','Albury']
qld = ['Gabba','Carrara','Cazaly\'s Stadium','Brisbane Exhibition']
sa = ['Football Park','Adelaide Oval']
wa = ['W.A.C.A.','Perth Stadium','Subiaco']
nt = ['Marrara Oval','Traeger Park']
tas = ['York Park','Bellerive Oval','North Hobart']
act = ['Manuka Oval','Bruce Stadium']
intl = ['Wellington','Jiangwan Stadium']

In [8]:
def locations(x):
    if x in nsw:
        return "NSW"
    elif x in sa:
        return "SA"
    elif x in wa:
        return "WA"
    elif x in nt:
        return "NT"
    elif x in tas:
        return "TAS"
    elif x in intl:
        return "INTL"
    elif x in act:
        return "ACT"
    elif x in qld:
        return "QLD"
    else: 
        return "VIC"

In [9]:
trans["state"] = trans["venue"].apply(locations)
trans["state"].value_counts()

VIC     13034
WA        641
SA        573
NSW       512
QLD       474
TAS        91
ACT        48
NT         24
INTL        5
Name: state, dtype: int64

In [10]:
## 1. HVAR & AGGR
trans['hvar'] = trans['pts_1'] - trans['pts_2']
trans['aggr'] = trans['pts_1'] + trans['pts_2']

In [11]:
## 2. YEAR & MONTH
import time
import datetime

In [12]:
trans['date'] = pd.to_datetime(trans['date'])
trans['year'], trans['month'] = trans['date'].dt.year, trans['date'].dt.month

In [13]:
## 3. ABBREVIATED CLUB NAMES
adelaide = ['Adelaide']
brisbane = ['Brisbane Bears','Brisbane Lions']
carlton = ['Carlton']
collingwood = ['Collingwood']
essendon = ['Essendon']
fitzroy = ['Fitzroy']
footscray = ['Footscray']
fremantle = ['Fremantle']
geelong = ['Geelong']
gold_coast = ['Gold Coast']
gws = ['GW Sydney']
hawthorn = ['Hawthorn']
melbourne = ['Melbourne']
north_melbourne = ['North Melbourne','Kangaroos']
port_adelaide = ['Port Adelaide']
richmond = ['Richmond']
south_melbourne = ['South Melbourne']
st_kilda = ['St Kilda']
sydney = ['Sydney']
west_coast = ['West Coast']
western_bulldogs = ['Western Bulldogs', 'Footscray']
university = ['University']

In [14]:
def team_shorten(x):
    if x in adelaide:
        return "AD"
    elif x in brisbane:
        return "BR"
    elif x in carlton:
        return "CA"
    elif x in collingwood:
        return "CO"
    elif x in essendon:
        return "ES"
    elif x in fitzroy:
        return "FI"
    elif x in fremantle:
        return "FR"
    elif x in geelong:
        return "GE"
    elif x in gold_coast:
        return "GC"
    elif x in gws:
        return "GW"
    elif x in hawthorn:
        return "HA"
    elif x in melbourne:
        return "ME"
    elif x in north_melbourne:
        return "NM"
    elif x in port_adelaide:
        return "PA"
    elif x in richmond:
        return "RI"
    elif x in south_melbourne:
        return "SM"
    elif x in st_kilda:
        return "SK"
    elif x in sydney:
        return "SY"
    elif x in west_coast:
        return "WC"
    elif x in western_bulldogs:
        return "WB"
    elif x in university:
        return "UN"
    else: 
        return "NOT_FOUND"

In [15]:
trans["t1"] = trans["team1"].apply(team_shorten)
trans["t2"] = trans["team2"].apply(team_shorten)

In [16]:
trans['t1'].value_counts().sum()

15402

##  FIX TEAM NAMES ANOMOLIES

* Merge Footscray & Western Bulldogs as **Western Bulldogs**
* Merge North Melbourne with Kangaroos as **North Melbourne**
* Merge Brisbane Lions with Brisbane Bears as **Brisbane**
* Leave Sydney and **South Melbourne** as separate
* Leave **Fitzroy** as separate
* Modify GW Sydney to **GWS Giants**

In [17]:
# Function from Analytics Vidhya
def coding(col, codeDict):
  colCoded = pd.Series(col, copy=True)
  for key, value in codeDict.items():
    colCoded.replace(key, value, inplace=True)
  return colCoded

In [18]:
trans["team1"] = coding(trans["team1"], {'Footscray':'Western Bulldogs'})
trans["team2"] = coding(trans["team2"], {'Footscray':'Western Bulldogs'})
trans["team1"] = coding(trans["team1"], {'Kangaroos':'North Melbourne'})
trans["team2"] = coding(trans["team2"], {'Kangaroos':'North Melbourne'})
trans["team1"] = coding(trans["team1"], {'Brisbane Bears':'Brisbane','Brisbane Lions':'Brisbane'})
trans["team2"] = coding(trans["team2"], {'Brisbane Bears':'Brisbane','Brisbane Lions':'Brisbane'})
trans["team1"] = coding(trans["team1"], {'GW Sydney':'GWS Giants'})
trans["team2"] = coding(trans["team2"], {'GW Sydney':'GWS Giants'})

In [19]:
trans["team2"].value_counts()

Collingwood         1274
Carlton             1251
Essendon            1222
Geelong             1219
St Kilda            1190
Melbourne           1190
Richmond            1118
North Melbourne      986
Hawthorn             979
Western Bulldogs     975
Fitzroy              960
South Melbourne      788
Sydney               432
West Coast           376
Brisbane             359
Adelaide             325
Fremantle            273
Port Adelaide        254
Gold Coast            88
GWS Giants            80
University            63
Name: team2, dtype: int64

#### The dataframe with the additional column. Currently it has one game per row.

In [20]:
trans.head(10)

,seq,date,rnd,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,pts_2,venue,state,hvar,aggr,year,month,t1,t2
1,1,1897-05-08,R1,Fitzroy,6,13,49,Carlton,2,4,16,Brunswick St,VIC,33,65,1897,5,FI,CA
2,2,1897-05-08,R1,Collingwood,5,11,41,St Kilda,2,4,16,Victoria Park,VIC,25,57,1897,5,CO,SK
3,3,1897-05-08,R1,Geelong,3,6,24,Essendon,7,5,47,Corio Oval,VIC,-23,71,1897,5,GE,ES
4,4,1897-05-08,R1,South Melbourne,3,9,27,Melbourne,6,8,44,Lake Oval,VIC,-17,71,1897,5,SM,ME
5,5,1897-05-15,R2,South Melbourne,6,4,40,Carlton,5,6,36,Lake Oval,VIC,4,76,1897,5,SM,CA
6,6,1897-05-15,R2,Essendon,4,6,30,Collingwood,8,2,50,East Melbourne,VIC,-20,80,1897,5,ES,CO
7,7,1897-05-15,R2,St Kilda,3,8,26,Fitzroy,10,6,66,Junction Oval,VIC,-40,92,1897,5,SK,FI
8,8,1897-05-15,R2,Melbourne,9,10,64,Geelong,3,1,19,M.C.G.,VIC,45,83,1897,5,ME,GE
9,9,1897-05-22,R3,Collingwood,6,5,41,Geelong,5,7,37,Victoria Park,VIC,4,78,1897,5,CO,GE
10,10,1897-05-22,R3,Fitzroy,5,9,39,Melbourne,7,8,50,Brunswick St,VIC,-11,89,1897,5,FI,ME


## ALL DRAWS (BOOLEAN)

In [21]:
def reg_draw(hvar):
    if hvar == 0:
        return 1
    else: return 0

In [22]:
trans["draw_rt"] = trans["hvar"].apply(reg_draw)

In [23]:
trans['draw_rt'].value_counts()

0    15244
1      158
Name: draw_rt, dtype: int64

### The 158 draws above, represent the VFL/AFL games that have concluded in a draw result

### The two ET (extra-time) draws below are statistically relevent, since they also ended regular full-time with both teams on the same score. However extra-time was applied to create a result, due to finals constraints.

In [24]:
def et_draw(seq):
    if seq == 10794:
        return 1
    elif seq == 13203:
        return 1
    else: return 0

In [25]:
trans["etime_draw"] = trans["seq"].apply(et_draw)
trans['etime_draw'].value_counts()

0    15400
1        2
Name: etime_draw, dtype: int64

In [26]:
### Add the two columns
trans['draws_all']  = trans['draw_rt'] + trans['etime_draw']
trans['draws_all'].value_counts()

0    15242
1      160
Name: draws_all, dtype: int64

In [27]:
# Check
trans.iloc[:,:][(trans.etime_draw == 1)]

,seq,date,rnd,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,...,state,hvar,aggr,year,month,t1,t2,draw_rt,etime_draw,draws_all
10794,10794,1994-09-10,QF,North Melbourne,15,24,114,Hawthorn,13,13,...,VIC,23,205,1994,9,NM,HA,0,1,1
13203,13203,2007-09-14,SF,Collingwood,13,15,93,West Coast,10,14,...,WA,19,167,2007,9,CO,WC,0,1,1


In [28]:
# 6. GAME CATEGORY
finals = ['EF','QF','SF','PF','GF']

In [29]:
def game_cat(x):
    if x in finals:
        return "final"
    else: 
        return "reg_season"

In [30]:
trans["type"] = trans["rnd"].apply(game_cat)

In [31]:
trans['type'].value_counts()

reg_season    14752
final           650
Name: type, dtype: int64

### So, there have been 650 finals up to the end of week one of 2018 finals series.

In [32]:
def grouping_2K(seq):
    n=500
    if seq <= 4*n:
        return 1
    elif seq <= 8*n:
        return 2
    elif seq <= 12*n:
        return 3
    elif seq <= 16*n:
        return 4
    elif seq <= 20*n:
        return 5
    elif seq <= 24*n:
        return 6
    elif seq <= 28*n:
        return 7
    else: return 8

In [33]:
trans["grp_2K"] = trans["seq"].apply(grouping_2K)

In [34]:
def grouping_1K(seq):
    n=1000
    if seq <= n:
        return 1
    elif seq <= 2*n:
        return 2
    elif seq <= 3*n:
        return 3
    elif seq <= 4*n:
        return 4
    elif seq <= 5*n:
        return 5
    elif seq <= 6*n:
        return 6
    elif seq <= 7*n:
        return 7
    elif seq <= 8*n:
        return 8
    elif seq <= 9*n:
        return 9
    elif seq <= 10*n:
        return 10
    elif seq <= 11*n:
        return 11
    elif seq <= 12*n:
        return 12
    elif seq <= 13*n:
        return 13
    elif seq <= 14*n:
        return 14
    elif seq <= 15*n:
        return 15
    else: return 16

In [35]:
trans["grp_1K"] = trans["seq"].apply(grouping_1K)

In [36]:
# Export the data frame for use in afl draws EDA
trans.to_csv('all_afl_wide.csv')

## ADD res1 and res2 columns as categorical 1 = win, 2 = loss, 3 = draw based on hvar (ignoring the two extra time finals)

In [37]:
def result_cat_h(hvar):
    if hvar > 0:
        return 1
    elif hvar < 0:
        return 2
    else: return 3

In [38]:
def result_cat_a(hvar):
    if hvar > 0:
        return 2
    elif hvar < 0:
        return 1
    else: return 3

In [39]:
trans['res1'] = trans['hvar'].apply(result_cat_h)
trans['res2'] = trans['hvar'].apply(result_cat_a)

In [40]:
trans.tail(9)

,seq,date,rnd,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,...,t1,t2,draw_rt,etime_draw,draws_all,type,grp_2K,grp_1K,res1,res2
15394,15394,2018-08-25,R23,Carlton,8,13,61,Adelaide,26,9,...,CA,AD,0,0,0,reg_season,8,16,2,1
15395,15395,2018-08-25,R23,Sydney,10,14,74,Hawthorn,12,11,...,SY,HA,0,0,0,reg_season,8,16,2,1
15396,15396,2018-08-26,R23,Brisbane,11,6,72,West Coast,14,14,...,BR,WC,0,0,0,reg_season,8,16,2,1
15397,15397,2018-08-26,R23,Melbourne,15,12,102,GWS Giants,8,9,...,ME,GW,0,0,0,reg_season,8,16,1,2
15398,15398,2018-08-26,R23,St Kilda,14,10,94,North Melbourne,17,15,...,SK,NM,0,0,0,reg_season,8,16,2,1
15399,15399,2018-09-06,QF,Richmond,13,17,95,Hawthorn,9,10,...,RI,HA,0,0,0,final,8,16,1,2
15400,15400,2018-09-07,EF,Melbourne,10,15,75,Geelong,6,10,...,ME,GE,0,0,0,final,8,16,1,2
15401,15401,2018-09-08,EF,GWS Giants,10,19,79,Sydney,4,6,...,GW,SY,0,0,0,final,8,16,1,2
15402,15402,2018-09-08,QF,West Coast,12,14,86,Collingwood,10,10,...,WC,CO,0,0,0,final,8,16,1,2


In [41]:
trans.shape

(15402, 27)

### Export enhanced results 1897-2018 with 27 columns

In [42]:
trans.to_csv('enh_1_aflresults.csv')

## STAGE TWO
## 9. Reduce and re-order the data frame to prepare for for Long format

In [43]:
tracker = trans
tracker = tracker[['year', 'rnd','type','team1','pts_1','team2','pts_2','venue','t1','t2','hvar']]
tracker.tail()

,year,rnd,type,team1,pts_1,team2,pts_2,venue,t1,t2,hvar
15398,2018,R23,reg_season,St Kilda,94,North Melbourne,117,Docklands,SK,NM,-23
15399,2018,QF,final,Richmond,95,Hawthorn,64,M.C.G.,RI,HA,31
15400,2018,EF,final,Melbourne,75,Geelong,46,M.C.G.,ME,GE,29
15401,2018,EF,final,GWS Giants,79,Sydney,30,S.C.G.,GW,SY,49
15402,2018,QF,final,West Coast,86,Collingwood,70,Perth Stadium,WC,CO,16


### Create two new columns with values 1 and 0 for the two subsets

In [44]:
tracker['subset_1'] = 1
tracker['subset_2'] = 0

/Applications/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Applications/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [45]:
tracker.tail(9)

,year,rnd,type,team1,pts_1,team2,pts_2,venue,t1,t2,hvar,subset_1,subset_2
15394,2018,R23,reg_season,Carlton,61,Adelaide,165,Docklands,CA,AD,-104,1,0
15395,2018,R23,reg_season,Sydney,74,Hawthorn,83,S.C.G.,SY,HA,-9,1,0
15396,2018,R23,reg_season,Brisbane,72,West Coast,98,Gabba,BR,WC,-26,1,0
15397,2018,R23,reg_season,Melbourne,102,GWS Giants,57,M.C.G.,ME,GW,45,1,0
15398,2018,R23,reg_season,St Kilda,94,North Melbourne,117,Docklands,SK,NM,-23,1,0
15399,2018,QF,final,Richmond,95,Hawthorn,64,M.C.G.,RI,HA,31,1,0
15400,2018,EF,final,Melbourne,75,Geelong,46,M.C.G.,ME,GE,29,1,0
15401,2018,EF,final,GWS Giants,79,Sydney,30,S.C.G.,GW,SY,49,1,0
15402,2018,QF,final,West Coast,86,Collingwood,70,Perth Stadium,WC,CO,16,1,0


In [46]:
droplist_1 = ['subset_2']
droplist_2 = ['subset_1']

In [47]:
left_team = tracker.drop(droplist_1, axis = 1)
right_team = tracker.drop(droplist_2, axis = 1)

In [48]:
left_team.shape

(15402, 12)

In [49]:
right_team.shape

(15402, 12)

# Rename columns in each dframe 

In [50]:
left_team.columns = ['year','rnd','type','team','tm_pts','opp_team','op_pts','venue','tm','op','hvar','lr']
right_team.columns = ['year','rnd','type','opp_team','op_pts','team','tm_pts','venue','op','tm','hvar','lr']

### Now re-order the second dframe to match columns

### Note double brackets to re-order

In [51]:
right_team = right_team[['year', 'rnd','type','team','tm_pts','opp_team','op_pts','venue','tm','op','hvar','lr']]
right_team.tail()

,year,rnd,type,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15398,2018,R23,reg_season,North Melbourne,117,St Kilda,94,Docklands,NM,SK,-23,0
15399,2018,QF,final,Hawthorn,64,Richmond,95,M.C.G.,HA,RI,31,0
15400,2018,EF,final,Geelong,46,Melbourne,75,M.C.G.,GE,ME,29,0
15401,2018,EF,final,Sydney,30,GWS Giants,79,S.C.G.,SY,GW,49,0
15402,2018,QF,final,Collingwood,70,West Coast,86,Perth Stadium,CO,WC,16,0


In [52]:
right_team.tail()

,year,rnd,type,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15398,2018,R23,reg_season,North Melbourne,117,St Kilda,94,Docklands,NM,SK,-23,0
15399,2018,QF,final,Hawthorn,64,Richmond,95,M.C.G.,HA,RI,31,0
15400,2018,EF,final,Geelong,46,Melbourne,75,M.C.G.,GE,ME,29,0
15401,2018,EF,final,Sydney,30,GWS Giants,79,S.C.G.,SY,GW,49,0
15402,2018,QF,final,Collingwood,70,West Coast,86,Perth Stadium,CO,WC,16,0


In [53]:
left_team.tail()

,year,rnd,type,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15398,2018,R23,reg_season,St Kilda,94,North Melbourne,117,Docklands,SK,NM,-23,1
15399,2018,QF,final,Richmond,95,Hawthorn,64,M.C.G.,RI,HA,31,1
15400,2018,EF,final,Melbourne,75,Geelong,46,M.C.G.,ME,GE,29,1
15401,2018,EF,final,GWS Giants,79,Sydney,30,S.C.G.,GW,SY,49,1
15402,2018,QF,final,West Coast,86,Collingwood,70,Perth Stadium,WC,CO,16,1


## CONCATENATION

### Can now concatenate the two dframes, as in si_week11

In [54]:
merge_ft = pd.concat([left_team, right_team], axis=0)
merge_ft.head()

,year,rnd,type,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
1,1897,R1,reg_season,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1
2,1897,R1,reg_season,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1
3,1897,R1,reg_season,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1
4,1897,R1,reg_season,South Melbourne,27,Melbourne,44,Lake Oval,SM,ME,-17,1
5,1897,R2,reg_season,South Melbourne,40,Carlton,36,Lake Oval,SM,CA,4,1


#### The new long data frame consisting of 30,000 plus rows:

In [55]:
merge_ft.shape

(30804, 12)

# The following is to keep each game together (two rows per game)

In [56]:
merge_ft.sort_index(inplace=True)
merge_ft.head(10)

,year,rnd,type,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
1,1897,R1,reg_season,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1
1,1897,R1,reg_season,Carlton,16,Fitzroy,49,Brunswick St,CA,FI,33,0
2,1897,R1,reg_season,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1
2,1897,R1,reg_season,St Kilda,16,Collingwood,41,Victoria Park,SK,CO,25,0
3,1897,R1,reg_season,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1
3,1897,R1,reg_season,Essendon,47,Geelong,24,Corio Oval,ES,GE,-23,0
4,1897,R1,reg_season,South Melbourne,27,Melbourne,44,Lake Oval,SM,ME,-17,1
4,1897,R1,reg_season,Melbourne,44,South Melbourne,27,Lake Oval,ME,SM,-17,0
5,1897,R2,reg_season,South Melbourne,40,Carlton,36,Lake Oval,SM,CA,4,1
5,1897,R2,reg_season,Carlton,36,South Melbourne,40,Lake Oval,CA,SM,4,0


In [57]:
merge_ft.tail()

,year,rnd,type,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15400,2018,EF,final,Melbourne,75,Geelong,46,M.C.G.,ME,GE,29,1
15401,2018,EF,final,Sydney,30,GWS Giants,79,S.C.G.,SY,GW,49,0
15401,2018,EF,final,GWS Giants,79,Sydney,30,S.C.G.,GW,SY,49,1
15402,2018,QF,final,West Coast,86,Collingwood,70,Perth Stadium,WC,CO,16,1
15402,2018,QF,final,Collingwood,70,West Coast,86,Perth Stadium,CO,WC,16,0


## There are actually 30,804 rows, indexing isn't  sequential

## Create team result column

In [58]:
def neg_hvar(hvar):
        return -1*hvar

In [59]:
merge_ft["neg_dummy"] = merge_ft['hvar'].apply(neg_hvar)

In [60]:
merge_ft.tail()

,year,rnd,type,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,neg_dummy
15400,2018,EF,final,Melbourne,75,Geelong,46,M.C.G.,ME,GE,29,1,-29
15401,2018,EF,final,Sydney,30,GWS Giants,79,S.C.G.,SY,GW,49,0,-49
15401,2018,EF,final,GWS Giants,79,Sydney,30,S.C.G.,GW,SY,49,1,-49
15402,2018,QF,final,West Coast,86,Collingwood,70,Perth Stadium,WC,CO,16,1,-16
15402,2018,QF,final,Collingwood,70,West Coast,86,Perth Stadium,CO,WC,16,0,-16


In [61]:
def team_var(hvar):
    if merge_ft['lr'] == 1:
        return hvar
    else: return neg_dummy

In [62]:
def new_lr(lr):
    if lr == 0:
        return -1
    else: return 1

In [63]:
merge_ft["new_lr"] = merge_ft['lr'].apply(new_lr)
merge_ft["tm_var"] = merge_ft['new_lr']*merge_ft['hvar']

In [64]:
merge_ft.tail()

,year,rnd,type,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,neg_dummy,new_lr,tm_var
15400,2018,EF,final,Melbourne,75,Geelong,46,M.C.G.,ME,GE,29,1,-29,1,29
15401,2018,EF,final,Sydney,30,GWS Giants,79,S.C.G.,SY,GW,49,0,-49,-1,-49
15401,2018,EF,final,GWS Giants,79,Sydney,30,S.C.G.,GW,SY,49,1,-49,1,49
15402,2018,QF,final,West Coast,86,Collingwood,70,Perth Stadium,WC,CO,16,1,-16,1,16
15402,2018,QF,final,Collingwood,70,West Coast,86,Perth Stadium,CO,WC,16,0,-16,-1,-16


### Can now remove the columns new_lr and neg_dummy, which were used to create tm_var

In [65]:
merge_ft = merge_ft.drop(['neg_dummy', 'new_lr'], axis=1)

In [66]:
merge_ft.head()

,year,rnd,type,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var
1,1897,R1,reg_season,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1,33
1,1897,R1,reg_season,Carlton,16,Fitzroy,49,Brunswick St,CA,FI,33,0,-33
2,1897,R1,reg_season,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1,25
2,1897,R1,reg_season,St Kilda,16,Collingwood,41,Victoria Park,SK,CO,25,0,-25
3,1897,R1,reg_season,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1,-23


### Note that the column 'lr' denotes left right, so there are only two different values (1 denotes left)

In [67]:
merge_ft['lr'].value_counts()

1    15402
0    15402
Name: lr, dtype: int64

## New function for win/loss/draw (W/L/D)


In [68]:
def outcome(tm_var):
    if tm_var >= 1:
        return "W"
    elif tm_var <= -1:
        return "L"
    elif tm_var == 0:
        return "D"
    else: return "Unknown"

In [69]:
merge_ft["outcome"] = merge_ft['tm_var'].apply(outcome)

In [70]:
merge_ft.head()

,year,rnd,type,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome
1,1897,R1,reg_season,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1,33,W
1,1897,R1,reg_season,Carlton,16,Fitzroy,49,Brunswick St,CA,FI,33,0,-33,L
2,1897,R1,reg_season,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1,25,W
2,1897,R1,reg_season,St Kilda,16,Collingwood,41,Victoria Park,SK,CO,25,0,-25,L
3,1897,R1,reg_season,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1,-23,L


In [71]:
merge_ft.tail(9)

,year,rnd,type,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome
15398,2018,R23,reg_season,St Kilda,94,North Melbourne,117,Docklands,SK,NM,-23,1,-23,L
15399,2018,QF,final,Hawthorn,64,Richmond,95,M.C.G.,HA,RI,31,0,-31,L
15399,2018,QF,final,Richmond,95,Hawthorn,64,M.C.G.,RI,HA,31,1,31,W
15400,2018,EF,final,Geelong,46,Melbourne,75,M.C.G.,GE,ME,29,0,-29,L
15400,2018,EF,final,Melbourne,75,Geelong,46,M.C.G.,ME,GE,29,1,29,W
15401,2018,EF,final,Sydney,30,GWS Giants,79,S.C.G.,SY,GW,49,0,-49,L
15401,2018,EF,final,GWS Giants,79,Sydney,30,S.C.G.,GW,SY,49,1,49,W
15402,2018,QF,final,West Coast,86,Collingwood,70,Perth Stadium,WC,CO,16,1,16,W
15402,2018,QF,final,Collingwood,70,West Coast,86,Perth Stadium,CO,WC,16,0,-16,L


### Add the state column back in, using locations function.

In [72]:
merge_ft["state"] = merge_ft["venue"].apply(locations)
merge_ft["state"].value_counts()

VIC     26068
WA       1282
SA       1146
NSW      1024
QLD       948
TAS       182
ACT        96
NT         48
INTL       10
Name: state, dtype: int64

### Export enhanced results v2 with long format two rows per match

In [73]:
merge_ft.to_csv('enh_2_aflresults_long.csv')

## Research for Week 2 of finals 2018 using Pandas filtering:
### * Hawthorn v Melbourne
### * Collingwood v GWS Giants 

### Recent contests Hawthorn v Melbourne

In [74]:
merge_ft.iloc[:,:][(merge_ft.tm == 'ME') & (merge_ft.op == 'HA')].tail(6)

,year,rnd,type,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome,state
14539,2014,R20,reg_season,Melbourne,65,Hawthorn,115,M.C.G.,ME,HA,50,0,-50,L,VIC
14637,2015,R7,reg_season,Melbourne,50,Hawthorn,155,M.C.G.,ME,HA,105,0,-105,L,VIC
14878,2016,R11,reg_season,Melbourne,64,Hawthorn,82,M.C.G.,ME,HA,18,0,-18,L,VIC
14951,2016,R20,reg_season,Melbourne,110,Hawthorn,81,M.C.G.,ME,HA,29,1,29,W,VIC
15055,2017,R7,reg_season,Melbourne,91,Hawthorn,94,M.C.G.,ME,HA,-3,1,-3,L,VIC
15235,2018,R4,reg_season,Melbourne,48,Hawthorn,115,M.C.G.,ME,HA,67,0,-67,L,VIC


### Hawks recent form

In [75]:
merge_ft.iloc[:,:][(merge_ft.tm == 'HA')].tail(10)

,year,rnd,type,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome,state
15323,2018,R15,reg_season,Hawthorn,84,GWS Giants,95,Sydney Showground,HA,GW,11,0,-11,L,NSW
15331,2018,R16,reg_season,Hawthorn,122,Western Bulldogs,59,Docklands,HA,WB,-63,0,63,W,VIC
15338,2018,R17,reg_season,Hawthorn,65,Brisbane,98,York Park,HA,BR,-33,1,-33,L,TAS
15351,2018,R18,reg_season,Hawthorn,124,Carlton,52,Docklands,HA,CA,-72,0,72,W,VIC
15362,2018,R19,reg_season,Hawthorn,112,Fremantle,53,Perth Stadium,HA,FR,-59,0,59,W,WA
15364,2018,R20,reg_season,Hawthorn,107,Essendon,103,M.C.G.,HA,ES,4,1,4,W,VIC
15373,2018,R21,reg_season,Hawthorn,71,Geelong,60,M.C.G.,HA,GE,11,1,11,W,VIC
15385,2018,R22,reg_season,Hawthorn,80,St Kilda,76,Docklands,HA,SK,-4,0,4,W,VIC
15395,2018,R23,reg_season,Hawthorn,83,Sydney,74,S.C.G.,HA,SY,-9,0,9,W,NSW
15399,2018,QF,final,Hawthorn,64,Richmond,95,M.C.G.,HA,RI,31,0,-31,L,VIC


### Recent contests Collingwood v GWS in Melbourne

In [76]:
merge_ft.iloc[:,:][(merge_ft.tm == 'CO') & (merge_ft.op == 'GW')].tail(12)

,year,rnd,type,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome,state
14107,2012,R18,reg_season,Collingwood,174,GWS Giants,54,Sydney Showground,CO,GW,-120,0,120,W,NSW
14314,2013,R18,reg_season,Collingwood,106,GWS Giants,66,M.C.G.,CO,GW,40,1,40,W,VIC
14557,2014,R22,reg_season,Collingwood,75,GWS Giants,67,Sydney Showground,CO,GW,-8,0,8,W,NSW
14675,2015,R11,reg_season,Collingwood,119,GWS Giants,77,M.C.G.,CO,GW,42,1,42,W,VIC
14915,2016,R16,reg_season,Collingwood,125,GWS Giants,93,Sydney Showground,CO,GW,-32,0,32,W,NSW
15060,2017,R8,reg_season,Collingwood,99,GWS Giants,102,Sydney Showground,CO,GW,3,0,-3,L,NSW
15213,2018,R2,reg_season,Collingwood,79,GWS Giants,95,M.C.G.,CO,GW,-16,1,-16,L,VIC


In [77]:
merge_ft.iloc[:,:][(merge_ft.tm == 'CO') & (merge_ft.op == 'GW') & (merge_ft.venue == 'M.C.G.')].tail(6)

,year,rnd,type,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome,state
14314,2013,R18,reg_season,Collingwood,106,GWS Giants,66,M.C.G.,CO,GW,40,1,40,W,VIC
14675,2015,R11,reg_season,Collingwood,119,GWS Giants,77,M.C.G.,CO,GW,42,1,42,W,VIC
15213,2018,R2,reg_season,Collingwood,79,GWS Giants,95,M.C.G.,CO,GW,-16,1,-16,L,VIC


### Collingwood at the MCG in 2018

In [78]:
merge_ft.iloc[:,:][(merge_ft.tm == 'CO') & (merge_ft.venue == 'M.C.G.')&(merge_ft.year == 2018) ]

,year,rnd,type,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome,state
15205,2018,R1,reg_season,Collingwood,67,Hawthorn,101,M.C.G.,CO,HA,34,0,-34,L,VIC
15213,2018,R2,reg_season,Collingwood,79,GWS Giants,95,M.C.G.,CO,GW,-16,1,-16,L,VIC
15219,2018,R3,reg_season,Collingwood,100,Carlton,76,M.C.G.,CO,CA,-24,0,24,W,VIC
15245,2018,R5,reg_season,Collingwood,101,Essendon,52,M.C.G.,CO,ES,49,1,49,W,VIC
15253,2018,R6,reg_season,Collingwood,70,Richmond,113,M.C.G.,CO,RI,-43,1,-43,L,VIC
15272,2018,R8,reg_season,Collingwood,45,Geelong,66,M.C.G.,CO,GE,-21,1,-21,L,VIC
15297,2018,R11,reg_season,Collingwood,138,Fremantle,77,M.C.G.,CO,FR,61,1,61,W,VIC
15305,2018,R12,reg_season,Collingwood,133,Melbourne,91,M.C.G.,CO,ME,-42,0,42,W,VIC
15317,2018,R14,reg_season,Collingwood,79,Carlton,59,M.C.G.,CO,CA,20,1,20,W,VIC
15334,2018,R16,reg_season,Collingwood,78,Essendon,62,M.C.G.,CO,ES,-16,0,16,W,VIC


In [79]:
merge_ft.iloc[:,:][(merge_ft.tm == 'CO') & (merge_ft.venue == 'M.C.G.')&(merge_ft.year == 2018)].median()

year      2018.0
tm_pts      79.0
op_pts      76.5
hvar         2.0
lr           1.0
tm_var      18.0
dtype: float64

### GWS' performances in Victoria: 2017 and 2018

In [80]:
merge_ft.iloc[:,:][(merge_ft.tm == 'GW') & (merge_ft.year >= 2017)&(merge_ft.state == 'VIC')]

,year,rnd,type,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome,state
15048,2017,R7,reg_season,GWS Giants,85,St Kilda,108,Docklands,GW,SK,23,0,-23,L,VIC
15094,2017,R12,reg_season,GWS Giants,70,Carlton,71,Docklands,GW,CA,1,0,-1,L,VIC
15144,2017,R18,reg_season,GWS Giants,45,Richmond,64,M.C.G.,GW,RI,19,0,-19,L,VIC
15165,2017,R21,reg_season,GWS Giants,105,Western Bulldogs,57,Docklands,GW,WB,-48,0,48,W,VIC
15187,2017,R23,reg_season,GWS Giants,59,Geelong,103,Kardinia Park,GW,GE,44,0,-44,L,VIC
15199,2017,PF,final,GWS Giants,67,Richmond,103,M.C.G.,GW,RI,36,0,-36,L,VIC
15213,2018,R2,reg_season,GWS Giants,95,Collingwood,79,M.C.G.,GW,CO,-16,0,16,W,VIC
15238,2018,R5,reg_season,GWS Giants,73,St Kilda,73,Docklands,GW,SK,0,0,0,D,VIC
15255,2018,R7,reg_season,GWS Giants,32,Geelong,93,Kardinia Park,GW,GE,61,0,-61,L,VIC
15369,2018,R20,reg_season,GWS Giants,151,Carlton,46,Docklands,GW,CA,-105,0,105,W,VIC


In [81]:
merge_ft.iloc[:,:][(merge_ft.tm == 'GW') & (merge_ft.year >= 2017)&(merge_ft.state == 'VIC')].median()

year      2017.0
tm_pts      70.0
op_pts      79.0
hvar        19.0
lr           0.0
tm_var     -19.0
dtype: float64

### GWS' outside NSW in 2018

In [82]:
merge_ft.iloc[:,:][(merge_ft.tm == 'GW') & (merge_ft.year > 2017)&(merge_ft.state != 'NSW')]

,year,rnd,type,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome,state
15207,2018,R1,reg_season,GWS Giants,133,Western Bulldogs,51,Manuka Oval,GW,WB,82,1,82,W,ACT
15213,2018,R2,reg_season,GWS Giants,95,Collingwood,79,M.C.G.,GW,CO,-16,0,16,W,VIC
15229,2018,R4,reg_season,GWS Giants,82,Fremantle,51,Manuka Oval,GW,FR,31,1,31,W,ACT
15238,2018,R5,reg_season,GWS Giants,73,St Kilda,73,Docklands,GW,SK,0,0,0,D,VIC
15255,2018,R7,reg_season,GWS Giants,32,Geelong,93,Kardinia Park,GW,GE,61,0,-61,L,VIC
15275,2018,R9,reg_season,GWS Giants,69,North Melbourne,112,Bellerive Oval,GW,NM,43,0,-43,L,TAS
15298,2018,R11,reg_season,GWS Giants,97,Adelaide,81,Adelaide Oval,GW,AD,-16,0,16,W,SA
15315,2018,R14,reg_season,GWS Giants,109,Brisbane,82,Gabba,GW,BR,-27,0,27,W,QLD
15335,2018,R16,reg_season,GWS Giants,75,West Coast,86,Perth Stadium,GW,WC,11,0,-11,L,WA
15353,2018,R18,reg_season,GWS Giants,80,Port Adelaide,58,Adelaide Oval,GW,PA,-22,0,22,W,SA


In [83]:
merge_ft.iloc[:,:][(merge_ft.tm == 'GW') & (merge_ft.year > 2017)&(merge_ft.state != 'NSW')].median()

year      2018.0
tm_pts      82.0
op_pts      81.0
hvar        11.0
lr           0.0
tm_var      16.0
dtype: float64

### Review Hawks finals performances since 2016

In [84]:
merge_ft.iloc[:,:][(merge_ft.tm == 'HA') & (merge_ft.year >= 2016)&(merge_ft.type == 'final')]

,year,rnd,type,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome,state
14986,2016,QF,final,Hawthorn,83,Geelong,85,M.C.G.,HA,GE,2,0,-2,L,VIC
14989,2016,SF,final,Hawthorn,84,Western Bulldogs,107,M.C.G.,HA,WB,23,0,-23,L,VIC
15399,2018,QF,final,Hawthorn,64,Richmond,95,M.C.G.,HA,RI,31,0,-31,L,VIC
